In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch


model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"


tokenizer = AutoTokenizer.from_pretrained(model_name)


bnb_config = BitsAndBytesConfig(load_in_4bit=True)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import load_dataset

dataset = load_dataset("/kaggle/input/finetunework1/finetunedataset")


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
from PIL import Image
import requests
from io import BytesIO

def preprocess_images(examples):
    images = []
    for img_data in examples['image']:
        
        if isinstance(img_data, str) and img_data.startswith('http'):
            response = requests.get(img_data)
            img = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            
            img = Image.open(img_data).convert("RGB")
        images.append(img)

    
    processed_images = [transform(img) for img in images]
    return {'pixel_values': processed_images}


In [8]:
import os

def preprocess_images(examples):
    images = []
    image_dir = '/kaggle/input/finetunework1/finetunedataset'
    for img_name in examples['image']:
        img_path = os.path.join(image_dir, img_name)  
        img = Image.open(img_path).convert("RGB")
        images.append(img)

    # Apply transformations to images
    processed_images = [transform(img) for img in images]
    return {'pixel_values': processed_images}
